In [6]:
####PREPROCESSING####
import pandas as pd 

data = pd.read_csv('songdata.csv.zip')

artists = set(data['artist'])
kanyetext = data.loc[data['artist'] == 'Kanye West','text']
#, 'Drake', 'Mc Hammer', 'Eminem', 'Migos', 'Flo-Rida'
draketext = data.loc[data['artist'] == 'Drake','text']
eminemtext = data.loc[data['artist'] == 'Eminem','text']
migostext = data.loc[data['artist'] == 'Migos', 'text']
floridatext = data.loc[data['artist'] == 'Flo-Rida', 'text']
mchammertext = data.loc[data['artist'] == 'Mc Hammer', 'text']

text = [kanyetext, draketext, eminemtext, migostext, floridatext, mchammertext]
text = pd.concat(text)
print(len(text))

385


In [7]:
text = ''.join(text) 
text = text.lower()
print(len(text))
print(text[:1000])

952008
why are you so paranoid?  
don't be so paranoid  
don't be so...  
  
baby, don't worry about it  
hey there, don't even think about it  
  
you worry bout the wrong things, the wrong things  
you worry bout the wrong things, the wrong things  
you worry bout the wrong things, the wrong things  
you worry bout the wrong things, the wrong things  
  
all of the time, you really wanna spend your whole life alone  
a little time out might do ya good, might do us good before you're done for
good  
because i could make it good, i could make it hood, i could make you come, i
could make you gooo  
i could make you hot, i could make you fly, make you touch the sky, hey, maybe
sooo  
all of the time, he be up in my, checking through my cell phone, baby no  
you wanna kill the vibe, on another night, here's another fight, oh, here we
go (oh, here we go)  
  
baby, don't worry 'bout it  
lady, we'll go out to the floor  
  
anyway, they don't know you like i do (they don't know)  
they'll 

In [8]:
#LSTM implementation adapted from https://github.com/fchollet/keras/blob/master/examples/lstm_text_generation.py

from __future__ import print_function
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys


chars = sorted(list(set(text))) #get all unique characters in the text
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars)) #map character to index
indices_char = dict((i, c) for i, c in enumerate(chars)) #map index to character

print(char_indices)

print(indices_char)

total chars: 50
{'\n': 0, ' ': 1, '!': 2, '"': 3, "'": 4, '(': 5, ')': 6, ',': 7, '-': 8, '.': 9, '0': 10, '1': 11, '2': 12, '3': 13, '4': 14, '5': 15, '6': 16, '7': 17, '8': 18, '9': 19, ':': 20, '?': 21, '[': 22, ']': 23, 'a': 24, 'b': 25, 'c': 26, 'd': 27, 'e': 28, 'f': 29, 'g': 30, 'h': 31, 'i': 32, 'j': 33, 'k': 34, 'l': 35, 'm': 36, 'n': 37, 'o': 38, 'p': 39, 'q': 40, 'r': 41, 's': 42, 't': 43, 'u': 44, 'v': 45, 'w': 46, 'x': 47, 'y': 48, 'z': 49}
{0: '\n', 1: ' ', 2: '!', 3: '"', 4: "'", 5: '(', 6: ')', 7: ',', 8: '-', 9: '.', 10: '0', 11: '1', 12: '2', 13: '3', 14: '4', 15: '5', 16: '6', 17: '7', 18: '8', 19: '9', 20: ':', 21: '?', 22: '[', 23: ']', 24: 'a', 25: 'b', 26: 'c', 27: 'd', 28: 'e', 29: 'f', 30: 'g', 31: 'h', 32: 'i', 33: 'j', 34: 'k', 35: 'l', 36: 'm', 37: 'n', 38: 'o', 39: 'p', 40: 'q', 41: 'r', 42: 's', 43: 't', 44: 'u', 45: 'v', 46: 'w', 47: 'x', 48: 'y', 49: 'z'}


In [9]:
text = [kanyetext, draketext, eminemtext, migostext, floridatext, mchammertext]
songs = []
for artist in text:
    for i in artist:
        songs.append(i.lower())
        
text = songs

In [10]:
####TRAINING MODEL####
# cut the text in semi-redundant sequences of maxlen characters, split by song

maxlen = 40
step = 3
sentences = []
next_chars = []

#breaks text up like this:     
'''["hello darkness my old friend i've come t", 
    "lo darkness my old friend i've come to t", 
    "darkness my old friend i've come to talk", 
    "kness my old friend i've come to talk wi", 
    "ss my old friend i've come to talk with ", 
    "my old friend i've come to talk with you", 
    "old friend i've come to talk with you ag"] --> sentences, and then puts the next char in next_chars'''

for i in range(len(text)): 
    for j in range(0, len(text[i]) - maxlen, step):
        sentences.append(text[i][j: j + maxlen])
        next_chars.append(text[i][j + maxlen])
print('number of sequences:', len(sentences))

number of sequences: 312329


In [11]:
print('Vectorization...')

#initialize two arrays, X = (maxlen rows x len(chars) cols) x len(sentences), y = len(sentences x len(chars))
X = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)

#for every sequence, mark the appearance of every char in that sequence and then mark the corresponding next char for that sequence
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        X[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

Vectorization...


In [12]:

# build the model: a single LSTM
print('Build model...')
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars))) 
model.add(Activation('softmax'))

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)


def sample(preds, temperature=1.0): #not really sure what temperature does
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

# train the model, output generated text after each iteration
for iteration in range(1, 60):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(X, y,
              batch_size=128,
              epochs=1)

    start_index = random.randint(0, len(text) - maxlen - 1)

    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print()
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index][start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x[0, t, char_indices[char]] = 1.

            preds = model.predict(x, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

Build model...

--------------------------------------------------
Iteration 1
Epoch 1/1
312329/312329 [==============================] - 584s - loss: 1.7935   

----- diversity: 0.2
----- Generating with seed: "t way too long  
i'm so gifted at findin"
t way too long  
i'm so gifted at finding to go  
i need to go  
i world the should the rone  
i wouldn the night  
i got to the never took the should we got  
i wanna never been the sking a shit  
i wouldn the for the should the close  
i wouldn the slow  
i know it too make the should the mone  
i got the never the what i got to the nother world  
i got to me  
i got the should the sking to love  
i wouldn the world the mone  
i would

----- diversity: 0.5
----- Generating with seed: "t way too long  
i'm so gifted at findin"
t way too long  
i'm so gifted at finding to dester  
i got's toper topers  
and i wanna got a should her can i know  
i'm say is girl i wanna gon' thinkin' up, the one  
  
when i would's everything a toruch  
i

can't a young nigga get money any more? lime  
he could wast up bark i crarnenis, yous he rap it  
your world watch it's just a do but i'm not we in made and brea  
what you want even jeys  
ovoon  
all over  
i know when they know you don't kill my drink lie gon side, i could have to see but the whatening camme heekets now my guity ig  
cause exen to get the izemes that you try it's probabel stars  
that to feel like your niggas bette

----- diversity: 1.2
----- Generating with seed: "
can't a young nigga get money any more?"

can't a young nigga get money any more? yo i'm liver, got grow (wassin' like i'm drar, nold)  
i do the devat beloes. nigga john  
what i goin' af!  
never (rebod yo bring andwe commip fol  
her mico.a)  
desixing,  
if you in allageream  
they get to weizin', jeast where you indeeze  
don't go over, goorder bull-thagu.ymbay  
  
trapped to them till used that pupissind down in howwer elseeaesly  
you ballas don't underson g-jid(band i

-------------------------

/home/lebronjames123/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:16: RuntimeWarning: divide by zero encountered in log
  app.launch_new_instance()


n' a gead my songs crima  
i won't know the son to do  
i live  
i 

----- diversity: 1.0
----- Generating with seed: "
can't a young nigga get money any more?"

can't a young nigga get money any more?  
do y't's brid that what  
counssa  
i know whoes been legs high  
where i was the all rane, what so bitches be yannoson  
oh yeah, kinds don't get down with the from jam the same lapped  
they kill your allurbed  
rell we live  
a funk on myt  
this weaking (oh) i work a from commen switch, i'd lad  
soate andmasty is homiends, never leave tonight spines aflerl to some lica for ampaniammy they 

----- diversity: 1.2
----- Generating with seed: "
can't a young nigga get money any more?"

can't a young nigga get money any more? so  
vid) i moves kanye, she alone for the ner dictges infews to  
i'm boney versame hims  
let ove that that's why stand to clab more?  
sleeve i got my, weat breath, (sheet from the ited fal my systla act would bit a, in the same done (ashud]  
and iclless, like

i can tell that good me  
i got a little cone and the streets  
i got a million is be so the fucking money  
i got the club  
i know you say i want to make it because  
i just stay the class me  
i got the club to the fucking money  
when i got the control  
i got a street all the life  
i know you wanna be the class  
i got the business i'm thinking  
the fuck the playing and the player  
i got the time  
  
[c

----- diversity: 0.5
----- Generating with seed: " waiting for me, oh ho  
i can tell that"
 waiting for me, oh ho  
i can tell that the was just that shit  
my face all of the time  
i know how many new motherfucke back the shit,  
i get me there will be a calling  
  
[verm attataftash]  
you said is the shit  
i don't mean and let me trying to be a world  
she contrortiled mine is my life  
can you head the night  
i was hammer  
i don't need a grink  
the shit that i don't really say  
it's  
yeah, the new come one hammer 

----- diversity: 1.0
----- Generating with seed: 

i'm chargilis flo ra?j i'm like and seen vivolly  
butting badd laumes  
jagpriess, what i've lost it tain, to gick,  
my raps to hite for you now i know  
down bla

----- diversity: 1.2
----- Generating with seed: "was a little bitty nigga in the hood,  
"
was a little bitty nigga in the hood,  
new ra?ja tellin', is how i ain't a tight  
 whitveliein nambaal be drees  
just mannator)  
i fuck thiskin' like ass ... not and  
xver too me like go  
and then it's never goes  
and emc so huffis top with an start ready  
i'm chance get right, ?s smalk clost it on wit's ask it to  
you drive lost with my lost on the sortal,  
if i be ggind...  
it's efrule to be  
before you keep  
ig with a ge

--------------------------------------------------
Iteration 15
Epoch 1/1
312329/312329 [==============================] - 703s - loss: 1.3738   

----- diversity: 0.2
----- Generating with seed: "mn girl i wanna ride with you  
waved up"
mn girl i wanna ride with you  
waved up the world i was the 

KeyboardInterrupt: 